In [0]:
# use this if you've just uploaded this notebook to Google Colaboratory
# don't forget to restart your runtime after the package installation
# better use a GPU runtime (TPU ones are not supported by the package yet)
!pip install neural-semigroups

If you have a Cayley database, you can build a machine learning model for such a task:

Given a partially filled Cayley table of a semigroup, restore the full one.

It should be mentioned that a partially filled table sometimes can be filled in several ways to a full associative table. We will consider all such solutions as equally valid.

In `neural-semigroups` package we use `torch` for building deep learning models.

First of all, we need to get some training and validation data.
In this example, we take semigroups of 5 items, and hold 100 Cayley tables (each representing a different class of equivalent semigrous) as our training data, and another 100 tables as validation.
This is a rough 10/90 split of all tables of 5 elements available (there are 1160 of them up to equivalence).

Here we construct `DataLoaders` for `torch` which will feed a training pipeline with 512 tables at a time.
This number (batch size) can be changed for fine-tuning the model's quality.

In [7]:
from neural_semigroups.training_helpers import get_loaders

cardinality = 5
data_loaders = get_loaders(
    cardinality=cardinality,
    batch_size=512,
    train_size=100,
    validation_size=100
)

generating test cubes: 100%|██████████| 960/960 [00:01<00:00, 947.69it/s]


Note that for a training set we:
* take 100 representatives of different equivalence classes
* augment data by adding all equivalent tables
* as a result, we will train on 16100 tables from 100 classes of equivalence

For validation we simply use 100 tables from different classes.

We model each input Cayley table as a three index tensor $a_{ijk}$ such that

$a_{ijk}=P\left\{e_ie_j=e_k\right\}$

where $e_i$ are elements of a semigroup.

In our training data all $a_{ijk}$ are either zeros or ones, so probability distributions involved are degenerate.

When we need to hide a cell with indices $i,j$ from an original Cayley table we set

$a_{ijk}=\dfrac1n$

where $n$ is the semigroup's cardinality. Thus we set a probability distribution of the multiplication result $e_ie_j$ to discrete uniform.

We choose a simple denoising autoencoder as an architecture for our neural network. It simply gets an input tensor of zeros and ones, hides 50% of input cells in a manner described earlier, and applies a linear transformation into a higher dimension ($n^5$ which is contrary to a common idea of autoencoders) with a simple `ReLU` non-linearity. Then another linear transformation to the same dimension with `ReLU` is applied, and then the last one to return back to the original $n^3$ dimension. We also apply batch normalization here. See the package code for the details.

In [0]:
from neural_semigroups import MagmaDAE
from neural_semigroups.constants import CURRENT_DEVICE

dae = MagmaDAE(
    cardinality=cardinality,
    hidden_dims=[
        cardinality ** 5,
        cardinality ** 5
    ],
    corruption_rate=0.5
)

In total, our model has ca 20 million  parameters.

In [9]:
sum(p.numel() for p in dae.parameters())

20341000

During the training process we try to minimize a special [associator loss](https://neural-semigroups.readthedocs.io/en/latest/package-documentation.html#associator-loss) on the output of the DAE.

In [0]:
import torch
from torch import Tensor
from neural_semigroups import AssociatorLoss

def loss(prediction: Tensor, target: Tensor) -> Tensor:
    return AssociatorLoss()(prediction)

Now it's time to run a pipeline! Here you can tune the learning schedule for better results.

You can construct your own pipeline if you don't want to import one provided by the package.

In the next three cells we will run `tensorboard` to show training/validation curves during training process.

In [0]:
%load_ext tensorboard

In [0]:
%tensorboard --logdir runs --host 0.0.0.0

In [13]:
from neural_semigroups.training_helpers import learning_pipeline

params = {"learning_rate": 0.001, "epochs": 1000}
learning_pipeline(params, cardinality, dae, loss, data_loaders)

Now we restore the best saved model from a checkpoint.

In [14]:
from neural_semigroups.utils import get_newest_file

dae.load_state_dict(torch.load(get_newest_file("checkpoints")))

<All keys matched successfully>

And here is the report of results. It seems to be quite impressive. For it we took random 1000 Cayley tables from 5 elements (for different equivalent classes as always) and constructed 'puzzles' from it.

Level of difficulty for a puzzle is a number of hidden cells. A puzzle is considered to be solved if the model returns a full associative table.

We see that the model generalizes well (it was trained only on one tenth of all equivalence classes).

In [19]:
from neural_semigroups.utils import print_report
from neural_semigroups import CayleyDatabase

cayley_db = CayleyDatabase(cardinality)
cayley_db.model = dae
print_report(cayley_db.testing_report())

generating and solving puzzles: 100%|██████████| 1000/1000 [04:06<00:00,  4.06it/s]


,puzzles,solved,(%),hidden cells,guessed,in %
level,,,,,,
1,1000.0,946.0,94.0,1000.0,946.0,94.0
2,1000.0,916.0,91.0,2000.0,1904.0,95.0
3,1000.0,888.0,88.0,3000.0,2844.0,94.0
4,1000.0,848.0,84.0,4000.0,3785.0,94.0
5,1000.0,837.0,83.0,5000.0,4730.0,94.0
6,1000.0,802.0,80.0,6000.0,5672.0,94.0
7,1000.0,792.0,79.0,7000.0,6596.0,94.0
8,1000.0,772.0,77.0,8000.0,7533.0,94.0
9,1000.0,766.0,76.0,9000.0,8446.0,93.0


As a sanity check we can look at the quality of a model which always fills in the same number, e.g. zero. Zero is a tangible choice since in the `smallsemi` database items are sorted in such a way that zero occurs most often. The NN model outperforms this baseline by a large margin.

In [20]:
from neural_semigroups.constant_baseline import ConstantBaseline

constant_baseline = CayleyDatabase(cardinality)
constant_baseline.model = ConstantBaseline(cardinality)
print_report(constant_baseline.testing_report())

generating and solving puzzles: 100%|██████████| 1000/1000 [02:39<00:00,  6.28it/s]


,puzzles,solved,(%),hidden cells,guessed,in %
level,,,,,,
1,1000.0,590.0,59.0,1000.0,590.0,59.0
2,1000.0,379.0,37.0,2000.0,1105.0,55.0
3,1000.0,308.0,30.0,3000.0,1675.0,55.0
4,1000.0,255.0,25.0,4000.0,2233.0,55.0
5,1000.0,206.0,20.0,5000.0,2759.0,55.0
6,1000.0,182.0,18.0,6000.0,3297.0,54.0
7,1000.0,164.0,16.0,7000.0,3833.0,54.0
8,1000.0,157.0,15.0,8000.0,4382.0,54.0
9,1000.0,143.0,14.0,9000.0,4838.0,53.0


Now let's see how it works on several example puzzles. Let's take one of the real tables from the database.

In [21]:
cayley_db.database[1100]

tensor([[0, 0, 0, 0, 0],
        [0, 1, 1, 1, 1],
        [0, 1, 2, 1, 1],
        [0, 1, 1, 3, 1],
        [0, 1, 1, 1, 4]], device='cuda:0')

Then we can fill it with `-1` in some cells, creating a puzzle and giving it to the model.

In [0]:
guess, proba = cayley_db.fill_in_with_model([
  [-1, 0, 0, 0, -1],
  [0, -1, 1, 1, -1],
  [0, 1, -1, 1, -1],
  [0, 1, 1, -1, -1],
  [0, 1, 1, 1, -1]]
)

The model found not the same table as the original one.

In [23]:
guess

tensor([[0, 0, 0, 0, 0],
        [0, 1, 1, 1, 1],
        [0, 1, 2, 1, 1],
        [0, 1, 1, 1, 1],
        [0, 1, 1, 1, 1]], device='cuda:0')

But it's still a possible completion since it's associative

In [24]:
from neural_semigroups import Magma

Magma(guess).is_associative

True

The model returns also it's probabilities of guess. They can be examined in cases when the model err.

In [25]:
proba

tensor([[[9.5899e-01, 2.1331e-02, 4.8319e-03, 9.6992e-03, 5.1428e-03],
         [1.0000e+00, 1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06],
         [1.0000e+00, 1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06],
         [1.0000e+00, 1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06],
         [9.9166e-01, 3.0912e-03, 1.1367e-03, 1.2539e-03, 2.8624e-03]],

        [[1.0000e+00, 1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06],
         [5.0471e-04, 9.9794e-01, 5.0806e-04, 5.3939e-04, 5.1256e-04],
         [1.0000e-06, 1.0000e+00, 1.0000e-06, 1.0000e-06, 1.0000e-06],
         [1.0000e-06, 1.0000e+00, 1.0000e-06, 1.0000e-06, 1.0000e-06],
         [1.6740e-03, 9.9021e-01, 1.6997e-03, 1.8921e-03, 4.5276e-03]],

        [[1.0000e+00, 1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06],
         [1.0000e-06, 1.0000e+00, 1.0000e-06, 1.0000e-06, 1.0000e-06],
         [4.9301e-03, 4.7933e-01, 5.0025e-01, 1.0046e-02, 5.4482e-03],
         [1.0000e-06, 1.0000e+00, 1.0000e-06, 1.0000e-06, 1.0000e-06],
  

In contrast, the constant baseline always fills in the `-1`s with zeros:

In [26]:
ans = constant_baseline.fill_in_with_model([
  [-1, 0, 0, 0, -1],
  [0, -1, 1, 1, -1],
  [0, 1, -1, 1, -1],
  [0, 1, 1, -1, -1],
  [0, 1, 1, 1, -1]]
)[0]
ans

tensor([[0, 0, 0, 0, 0],
        [0, 0, 1, 1, 0],
        [0, 1, 0, 1, 0],
        [0, 1, 1, 0, 0],
        [0, 1, 1, 1, 0]], device='cuda:0')

In [27]:
Magma(ans).is_associative

False